In [1]:
from TCSPC import *

In [2]:
EGFP = Simulation([0.497,0.503],[2.43,3.07])

In [33]:
tdata = np.loadtxt('EGFPt.csv')
ydata = np.loadtxt('EGFPy2.csv')

In [3]:
def trim_rescale_data(tdata,ydata,end =int(0.6*380)):
    max_idx = np.argmax(ydata) #index of data point with maximum photon count N(0)
    tdata = tdata[:end-max_idx] #start from t = 0
    ydata = ydata[max_idx:end]  #start from max.
    yerr = ydata/ydata[0]*np.sqrt(1/ydata+1/ydata[0]) #error after scaling
    ydata = ydata/ydata[0] # scale y data such that the beginning is 1 
    weights = 1/yerr #weighted by 1/yerr, yerr is error after scaling ydata
    return tdata,ydata

In [4]:
def exp(t, A, tau):
    return A * np.exp(-t/tau)

In [5]:
tdata,ydata = trim_rescale_data(EGFP.t,EGFP.y2)

In [6]:
def poisson_deviance_residual(observed, expected):
    # Ensure no invalid values
    valid = (observed > 0) & (expected > 0)
    residual = np.zeros_like(observed)
    residual[valid] = np.sign(observed[valid] - expected[valid]) * np.sqrt(
        2 * (observed[valid] * np.log(observed[valid] / expected[valid]) - (observed[valid] - expected[valid]))
    )
    return residual

def residual(p, t, data):
    v = p.valuesdict()
    generative = v['c'] #constant background
    M = 0
    while f'A{M}' in v:
        generative += exp(t, v[f'A{M}'], v[f'tau{M}'])
        M += 1
    
    return poisson_deviance_residual(data, generative)


In [15]:
def initial_peak_params(M,A_guess,tau_guess):
    p = lmfit.Parameters()
    p.add_many(('c', 0, True, 0, 1)) #constant background
    for i in range(M): #for each component
        p.add_many((f'A{i}', A_guess[i], True,0), #amplitude
                   (f'tau{i}', tau_guess[i], True, 0)) #lifetime
    p[f'A{M-1}'].set(expr = f'1-sum([A{i} for i in range({M}-1)])') #fix the amplitude of last component
    return p


In [16]:
p1 = initial_peak_params(2,EGFP.amp,EGFP.tau)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata), method='powell')

print(lmfit.fit_report(mi1))


RecursionError: maximum recursion depth exceeded while calling a Python object